## Trabajo Fin de Máster <br/> Diseño de una arquitectura multimodal para descripción textual de pares imagen-audio

## Script 3. Obtención de imágenes a partir de la base de datos

En este notebook, tomamos los vídeos que forman parte de la base de datos y extraemos los fotogramas que los componen. Para ello, hacemos uso de la biblioteca OpenCV.

### Paso 1. Montamos el almacenamiento

Damos permiso a Colab para acceder a mi unidad de Drive y nos situamos en la carpeta donde tenemos los scripts y la librería que hemos creado con las clases propias.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/TFM/Proyecto/Scripts

Mounted at /content/drive
/content/drive/MyDrive/TFM/Proyecto/Scripts


### Paso 2. Importación de paquetes

In [ ]:
import os
import cv2
import shutil
import pandas as pd

### Paso 3. Extracción de imágenes
Dentro de la carpeta correspondiente a cada participante, tenemos dos archivos: 'video_sq.mp4' y 'audio.ogg'. Tomamos el primero para convertirlo en imágenes con la ayuda de `cv2`.

Lo que hacemos es tomar el vídeo (de imagen cuadrada) y guardar cada uno de los frames como una imagen independiente de las demás. Con ello, acabamos teniendo una cantidad de imágenes del orden de 300-600 por persona.

In [ ]:
def ImageExtraction(db_path, new_db_path, output_dim=None):

    participants = os.listdir(db_path)
    if '.DS_Store' in participants:
        participants.remove('.DS_Store')

    rel_paths = []
    classID = []
    for person in participants:
        video = cv2.VideoCapture(f"{db_path}/{person}/video_sq.mp4")

        folder_path = f"{new_db_path}/image/{person}"
        try:
            # Creamos la carpetas donde guardaremos las imágenes, en caso de que no exista
            if os.path.exists(folder_path):
                shutil.rmtree(folder_path)
            os.makedirs(folder_path)
        except OSError:
            print ('Error: Creating directory of data')

        # Iniciamos la cuenta de fotogramas extraídos
        currentframe = 0

        while(True):

            # Leemos el fotograma. El método read() devuelve una tupla con dos valores:
            # ret. Booleano que muestra si la lectura fue exitosa (True/False)
            # frame. Numpy array (x,y,3) que contiene la imagen leída.
            ret, frame = video.read()

            if ret:
                # Crear imágenes mientras quede vídeo (ret=True)
                name = folder_path + '/frame{:0>5}.jpg'.format(currentframe)

                # Escritura del fotograma extraído
                resized_frame = cv2.resize(frame, output_dim) if output_dim else frame
                cv2.imwrite(name, resized_frame)
                classID.append(person)
                rel_paths.append(name)

                # Guardamos las dimensiones del primer frame para imprimirlo posteriormente.
                if currentframe == 0:
                    dims = resized_frame.shape

                # Incrementamos la cuenta de fotogramas
                currentframe += 1
            else:
                # Si ya no queda vídeo, salimos del bucle.
                break

        # Imprimimos información sobre el participante empleado.
        print(f"{person}: {currentframe} frames of {str(dims)}.")

    return pd.DataFrame({
        'image_path': rel_paths,
        'classID': classID
    })

imageDB_df = ImageExtraction("../Database_mini", "../Final_Database_mini")
display(imageDB_df)

Jose Alberto Azorin Puche: 605 frames of (480, 480, 3).
Noelia Sanchez Alonso: 439 frames of (480, 480, 3).
Maria Jose Morales Forte: 524 frames of (480, 480, 3).
Genesis Reyes Arteaga: 363 frames of (480, 480, 3).
Juan Cuesta Lopez: 455 frames of (480, 480, 3).
Juanjo Bautista Ibanez: 694 frames of (480, 480, 3).


,image_path,classID
0,../Final_Database_mini/image/Jose Alberto Azor...,Jose Alberto Azorin Puche
1,../Final_Database_mini/image/Jose Alberto Azor...,Jose Alberto Azorin Puche
2,../Final_Database_mini/image/Jose Alberto Azor...,Jose Alberto Azorin Puche
3,../Final_Database_mini/image/Jose Alberto Azor...,Jose Alberto Azorin Puche
4,../Final_Database_mini/image/Jose Alberto Azor...,Jose Alberto Azorin Puche
...,...,...
3075,../Final_Database_mini/image/Juanjo Bautista I...,Juanjo Bautista Ibanez
3076,../Final_Database_mini/image/Juanjo Bautista I...,Juanjo Bautista Ibanez
3077,../Final_Database_mini/image/Juanjo Bautista I...,Juanjo Bautista Ibanez
3078,../Final_Database_mini/image/Juanjo Bautista I...,Juanjo Bautista Ibanez


In [ ]:
imageDB_df.to_csv('../Final_Database_mini/image/imagesDB.csv', index=False)